## production_run_test

Writing and testing the functionality of scripts which will be used to carry out a 'production run,' where single-component Sérsic profiles will be fit to each galaxy in multiple filters via MCMC sampling.

In [4]:
# math, array manipulation, etc.
import numpy as np

import os
import sys
print(sys.version)

# timing
from timeit import default_timer
from tqdm import tqdm

import pandas as pd

import astropy.io.fits as fits
from astropy.table import Table                    # Table data structure
import astropy.units as u

# necessary utilities from scipy, astropy and photutils
from scipy.optimize import differential_evolution
from scipy.ndimage import maximum_filter, gaussian_filter
from astropy.modeling import functional_models
from astropy.convolution import convolve, Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma, SigmaClip
from photutils.background import Background2D, StdBackgroundRMS
from photutils.segmentation import deblend_sources, SegmentationImage, detect_sources

# plots
import matplotlib.pyplot as plt

#%matplotlib inline

# MCMC sampling package
import emcee

# SEP
import sep

3.10.4 (main, Mar 31 2022, 03:38:35) [Clang 12.0.0 ]


/Users/aidan/opt/anaconda3/envs/gal-gal-sel/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
sys.path.append('/Users/aidan/Desktop/sl_project/galaxy-galaxy-sel/')

from lensphoto import LensPhoto, fwhm2sigma

from aidan_utils import plot_prettier, read_fits_image, get_fits_files

# only need to run once
plot_prettier()


In [6]:
from astropy.io.fits import HDUList

from astroquery.vizier import Vizier

vsearch = Vizier

# extract table from the Jacobs, et al. (2019) catalog

vsearch.ROW_LIMIT = -1 # select all rows, not just default 50 first rows
vsearch.columns=['*']
#vsearch.columns=['Cand', 'objID', 'RAJ2000', 'DEJ2000', 'Grade', 'z']

sl_jacobs_catalog = vsearch.get_catalogs('J/ApJS/243/17/table1')
sl_table = sl_jacobs_catalog[0]

In [7]:
jr_path = '/Users/aidan/Desktop/sl_project/img_cutouts/sl_jacobs/rband_dr2/no_cluster_scale/mask_works/fit_works/'
jg_path = '/Users/aidan/Desktop/sl_project/img_cutouts/sl_jacobs/gband_dr2/no_cluster_scale/mask_works/fit_works/'

files_r = get_fits_files(jr_path)
files_g = get_fits_files(jg_path)

index = 19

cand, img_r = read_fits_image(files_r[index])
cand, img_g = read_fits_image(files_g[index])

#print(files_r[index])
#print(files_g[index])

# test putting multiple images into one HDU
# then try it out with the two
# also try to compile photometric redshift (with Vizier, see simple_csv_creator)

In [8]:
hdulist_r = HDUList()
hdulist_g = HDUList()

for i in np.arange(len(files_r)):
    
    file_r, file_g = files_r[i], files_g[i]
    
    hdu_r = fits.open(file_r)[0]
    hdu_g = fits.open(file_g)[0]
    
    cand = file_r[-34:-26] + file_r[-19:-14]
    
    hdu_r.header['cand'] = cand
    hdu_g.header['cand'] = cand
    
    assert (cand == file_g[-34:-26] + file_g[-19:-14])
    
    zi = sl_table[sl_table['Cand'] == cand]['z'][0]
    hdu_r.header['gphoto_z'] = zi
    hdu_g.header['gphoto_z'] = zi
    
    hdulist_r.append(hdu_r)
    hdulist_g.append(hdu_g)
    
hdulist_r.writeto('/Users/aidan/Desktop/sl_project/production/lens_images_r.fits', overwrite=True)
hdulist_g.writeto('/Users/aidan/Desktop/sl_project/production/lens_images_g.fits', overwrite=True)

#### Testing